### load the data

In [ ]:
!pip install git+https://github.com/qubvel/segmentation_models

In [ ]:
cd /content/drive/MyDrive/3033proj

/content/drive/.shortcut-targets-by-id/15vNI9If7Er0ikh22tgyBff1Q88-gfcL-/3033proj


In [ ]:
import keras
import os
import numpy as np

class Datasets(object):
    def __init__(self,data_name):
        self.data_name=data_name
        self.datasets = self.getData(self)
        self.final_data = []
        self.convert_data_format()


    def gen(self):
        # np.random.shuffle(self.final_data)

        images = []
        age_labels = []
        gender_labels = []

        for i in range(len(self.final_data)):
            image, age, gender = self.final_data[i]
            images.append(image)
            age_labels.append(age)
            gender_labels.append(gender)
    

        age_labels = keras.utils.to_categorical(age_labels, num_classes=NUM_AGE_CLASSES)
        gender_labels = keras.utils.to_categorical(gender_labels, num_classes=NUM_GENDER_CLASSES)
        return images, age_labels, gender_labels

    @staticmethod
    def getData(self):
        print('Loading age image...')
        # data_3
        if self.data_name=='data_3_utk':
          data = np.load(os.path.join(os.getcwd(), '/content/drive/MyDrive/3033proj/data_3_utk.npy'), allow_pickle=True) 
        if self.data_name=='data_5_utk':
          data = np.load(os.path.join(os.getcwd(), '/content/drive/MyDrive/3033proj/data_5_utk.npy'), allow_pickle=True) 
        if self.data_name=='data_10_utk':
          data = np.load(os.path.join(os.getcwd(), '/content/drive/MyDrive/3033proj/data_10_utk.npy'), allow_pickle=True)
        np.random.shuffle(data)
        all_data = []
        # random select 45000 data
        for i in range(20000):  ### number of samples
            all_data.append(data[i])

        print('Number of age data:', str(len(all_data)))

        return all_data

    def convert_data_format(self):
        # Age datasets:
        for i in range(len(self.datasets)):
            image = self.datasets[i][0] / 255.0
            age_labels = self.datasets[i][1]
            gender_labels = self.datasets[i][2]
            self.final_data.append((image, age_labels, gender_labels))

In [ ]:
from keras.callbacks import ReduceLROnPlateau, ModelCheckpoint
from keras import optimizers
from keras.utils.generic_utils import get_custom_objects
from keras.layers import GlobalMaxPool2D, Dropout, Dense, Activation, BatchNormalization
from keras.models import Model
import numpy as np
from keras.backend import sigmoid
from efficientnet.keras import EfficientNetB4 as NetB4
from efficientnet.keras import EfficientNetB4 as NetB5
from efficientnet.keras import EfficientNetB4 as NetB6
from efficientnet.keras import EfficientNetB4 as NetB7

### set parameters

In [ ]:
IMAGE_SIZE = 56
BATCH_SIZE = 256
WEIGHT_INIT = 0.08
NUM_AGE_CLASSES = 3
NUM_GENDER_CLASSES = 2
DROPOUT_RATE = 0.2
input_shape = (IMAGE_SIZE, IMAGE_SIZE, 3)
NUM_EPOCHS = 40
DECAY_LR_RATE = 0.9
NUM_SAMPLE=45000

## model

In [ ]:
import pandas as pd
class SwishActivation(Activation):
    def __init__(self, activation, **kwargs):
        super(SwishActivation, self).__init__(activation, **kwargs)
        self.__name__ = 'swish_act'

def swish_act(x, beta=1):
    return x * sigmoid(beta * x)

class Efficient_Net(object):
    def __init__(self, model_name, data_name, trainable=True):
        self.trainable = trainable
        self.model_name=model_name
        self.data_name=data_name
        if self.trainable: 
            self.train_data = Datasets(data_name=self.data_name)
        self.model = self.build_model(self.model_name)

        # Compile the model
        losses = {
            "age_output": "categorical_crossentropy",
            "gender_output": "categorical_crossentropy"
        }

        opt = optimizers.Adam(1e-3)
        self.model.compile(loss=losses, optimizer=opt, metrics=['acc'])

        # Train the part you added
        if self.trainable:
            self.model.summary()

    @staticmethod
    def build_age_branch(x):
        # Output age branch
        predictions_age = Dense(NUM_AGE_CLASSES, activation="softmax", name='age_output')(x)

        return predictions_age

    @staticmethod
    def build_gender_branch(x):
        # Output gender branch
        predictions_gender = Dense(NUM_GENDER_CLASSES, activation="softmax", name='gender_output')(x)

        return predictions_gender

    def build_model(self,model_name):
        get_custom_objects().update({'swish_act': SwishActivation(swish_act)})

        # Model
        if model_name=='age_gender_B4':
          model = NetB4(weights='imagenet', include_top=False, input_shape=input_shape)
        if model_name=='age_gender_B5':
          model = NetB5(weights='imagenet', include_top=False, input_shape=input_shape)
        if model_name=='age_gender_B6':
          model = NetB6(weights='imagenet', include_top=False, input_shape=input_shape)
        if model_name=='age_gender_B7':
          model = NetB7(weights='imagenet', include_top=False, input_shape=input_shape)
        

        # Adding 2 fully-connected layers to B4.
        x = model.output

        x = BatchNormalization()(x)
        x = GlobalMaxPool2D(name='gap1')(x)
        x = Dropout(DROPOUT_RATE, name='dropout1')(x)

        # Output layer
        predictions_age = self.build_age_branch(x)
        predictions_gender = self.build_gender_branch(x)
        model_final = Model(inputs=model.input, outputs=[predictions_age, predictions_gender])

        return model_final

    def train(self):
        # reduce learning rate
        reduce_lr = ReduceLROnPlateau(monitor='val_age_output_acc', factor=DECAY_LR_RATE, patience=5, verbose=1, )
        # Model Checkpoint

        trainX, trainAgeY, trainGenderY = self.train_data.gen()
        trainX = np.array(trainX)


        history=self.model.fit(trainX, {"age_output": trainAgeY, "gender_output": trainGenderY}, validation_split=0.2,
                       callbacks=reduce_lr, verbose=1, epochs=40, shuffle=True,
                       batch_size=256)
        
        dir ='/content/drive/MyDrive/3033proj/model_log/'
        log_path = dir+self.model_name+self.data_name+'_log.csv'
        hist = pd.DataFrame(history.history)
        hist.to_csv(log_path)


## age_gender_B4', 'data_3_ukt'

In [ ]:
NUM_AGE_CLASSES=3
model = Efficient_Net('age_gender_B4', 'data_3_utk', trainable=True)

In [ ]:
model.train('age_gender_B4', 'data_3_utk')

## age_gender_B5', 'data_3_utk'

In [ ]:
NUM_AGE_CLASSES=3
model = Efficient_Net('age_gender_B5', 'data_3_utk', trainable=True)

In [ ]:
model.train()

## age_gender_B6', 'data_3_utk'

In [ ]:
NUM_AGE_CLASSES=3
model = Efficient_Net('age_gender_B6', 'data_3_utk', trainable=True)

In [ ]:
model.train()

## age_gender_B7', 'data_3_utk'

In [ ]:
NUM_AGE_CLASSES=3
model = Efficient_Net('age_gender_B7', 'data_3_utk', trainable=True)

In [ ]:
model.train()

Epoch 1/40
63/63 [==============================] - 41s 326ms/step - loss: 2.5577 - age_output_loss: 1.6123 - gender_output_loss: 0.9454 - age_output_acc: 0.4799 - gender_output_acc: 0.6828 - val_loss: 1.6856 - val_age_output_loss: 0.9843 - val_gender_output_loss: 0.7013 - val_age_output_acc: 0.5123 - val_gender_output_acc: 0.7700
Epoch 2/40
63/63 [==============================] - 17s 273ms/step - loss: 1.2308 - age_output_loss: 0.8284 - gender_output_loss: 0.4024 - age_output_acc: 0.6456 - gender_output_acc: 0.8427 - val_loss: 1.3306 - val_age_output_loss: 0.9247 - val_gender_output_loss: 0.4059 - val_age_output_acc: 0.6313 - val_gender_output_acc: 0.8413
Epoch 3/40
63/63 [==============================] - 17s 273ms/step - loss: 0.9185 - age_output_loss: 0.6585 - gender_output_loss: 0.2600 - age_output_acc: 0.7077 - gender_output_acc: 0.8938 - val_loss: 1.1396 - val_age_output_loss: 0.7760 - val_gender_output_loss: 0.3636 - val_age_output_acc: 0.6835 - val_gender_output_acc: 0.8648
E

## age_gender_B4', 'data_5_utk'

In [ ]:
NUM_AGE_CLASSES=5
model = Efficient_Net('age_gender_B4', 'data_5_utk', trainable=True)

In [ ]:
model.train()

Epoch 1/40
63/63 [==============================] - 52s 330ms/step - loss: 3.1666 - age_output_loss: 2.1341 - gender_output_loss: 1.0324 - age_output_acc: 0.4028 - gender_output_acc: 0.6793 - val_loss: 1.7794 - val_age_output_loss: 1.2587 - val_gender_output_loss: 0.5207 - val_age_output_acc: 0.5285 - val_gender_output_acc: 0.7400
Epoch 2/40
63/63 [==============================] - 17s 270ms/step - loss: 1.4783 - age_output_loss: 1.1111 - gender_output_loss: 0.3672 - age_output_acc: 0.6122 - gender_output_acc: 0.8440 - val_loss: 1.6838 - val_age_output_loss: 1.2114 - val_gender_output_loss: 0.4724 - val_age_output_acc: 0.6018 - val_gender_output_acc: 0.8415
Epoch 3/40
63/63 [==============================] - 17s 270ms/step - loss: 1.1441 - age_output_loss: 0.8720 - gender_output_loss: 0.2721 - age_output_acc: 0.6777 - gender_output_acc: 0.8871 - val_loss: 1.2784 - val_age_output_loss: 0.9459 - val_gender_output_loss: 0.3325 - val_age_output_acc: 0.6205 - val_gender_output_acc: 0.8568
E

## age_gender_B5', 'data_5_utk'

In [ ]:
NUM_AGE_CLASSES=5
model = Efficient_Net('age_gender_B5', 'data_5_utk', trainable=True)

In [ ]:
model.train()

Epoch 1/40
63/63 [==============================] - 35s 308ms/step - loss: 2.8474 - age_output_loss: 2.0072 - gender_output_loss: 0.8401 - age_output_acc: 0.4176 - gender_output_acc: 0.7051 - val_loss: 1.7637 - val_age_output_loss: 1.2304 - val_gender_output_loss: 0.5334 - val_age_output_acc: 0.5505 - val_gender_output_acc: 0.7642
Epoch 2/40
63/63 [==============================] - 17s 269ms/step - loss: 1.5205 - age_output_loss: 1.1195 - gender_output_loss: 0.4010 - age_output_acc: 0.6097 - gender_output_acc: 0.8340 - val_loss: 1.5945 - val_age_output_loss: 1.0860 - val_gender_output_loss: 0.5085 - val_age_output_acc: 0.6072 - val_gender_output_acc: 0.7830
Epoch 3/40
63/63 [==============================] - 17s 269ms/step - loss: 1.2681 - age_output_loss: 0.9452 - gender_output_loss: 0.3229 - age_output_acc: 0.6539 - gender_output_acc: 0.8673 - val_loss: 1.4096 - val_age_output_loss: 1.0806 - val_gender_output_loss: 0.3290 - val_age_output_acc: 0.5853 - val_gender_output_acc: 0.8630
E

## age_gender_B6', 'data_5_utk'

In [ ]:
NUM_AGE_CLASSES=5
model = Efficient_Net('age_gender_B6', 'data_5_utk', trainable=True)

In [ ]:
model.train()

Epoch 1/40
63/63 [==============================] - 35s 306ms/step - loss: 2.9726 - age_output_loss: 2.0310 - gender_output_loss: 0.9416 - age_output_acc: 0.4070 - gender_output_acc: 0.6885 - val_loss: 1.6116 - val_age_output_loss: 1.1661 - val_gender_output_loss: 0.4455 - val_age_output_acc: 0.5780 - val_gender_output_acc: 0.7983
Epoch 2/40
63/63 [==============================] - 17s 270ms/step - loss: 1.4137 - age_output_loss: 1.0670 - gender_output_loss: 0.3467 - age_output_acc: 0.6138 - gender_output_acc: 0.8524 - val_loss: 1.4381 - val_age_output_loss: 1.0223 - val_gender_output_loss: 0.4159 - val_age_output_acc: 0.5922 - val_gender_output_acc: 0.8123
Epoch 3/40
63/63 [==============================] - 17s 270ms/step - loss: 1.2192 - age_output_loss: 0.9121 - gender_output_loss: 0.3072 - age_output_acc: 0.6437 - gender_output_acc: 0.8751 - val_loss: 1.3013 - val_age_output_loss: 0.9648 - val_gender_output_loss: 0.3365 - val_age_output_acc: 0.6308 - val_gender_output_acc: 0.8515
E

## age_gender_B7', 'data_5_utk'

In [ ]:
NUM_AGE_CLASSES=5
model = Efficient_Net('age_gender_B7', 'data_5_utk', trainable=True)

In [ ]:
model.train()

Epoch 1/40
63/63 [==============================] - 35s 310ms/step - loss: 3.0183 - age_output_loss: 2.1232 - gender_output_loss: 0.8952 - age_output_acc: 0.3996 - gender_output_acc: 0.6887 - val_loss: 3.8554 - val_age_output_loss: 2.2359 - val_gender_output_loss: 1.6195 - val_age_output_acc: 0.3738 - val_gender_output_acc: 0.7203
Epoch 2/40
63/63 [==============================] - 17s 270ms/step - loss: 1.4401 - age_output_loss: 1.0754 - gender_output_loss: 0.3647 - age_output_acc: 0.6176 - gender_output_acc: 0.8491 - val_loss: 3.0501 - val_age_output_loss: 2.6620 - val_gender_output_loss: 0.3881 - val_age_output_acc: 0.5552 - val_gender_output_acc: 0.8510
Epoch 3/40
63/63 [==============================] - 17s 270ms/step - loss: 1.1585 - age_output_loss: 0.9013 - gender_output_loss: 0.2572 - age_output_acc: 0.6628 - gender_output_acc: 0.8913 - val_loss: 1.3658 - val_age_output_loss: 1.0024 - val_gender_output_loss: 0.3634 - val_age_output_acc: 0.6277 - val_gender_output_acc: 0.8530
E

## age_gender_B4', 'data_10_utk'

In [ ]:
NUM_AGE_CLASSES=10
model = Efficient_Net('age_gender_B4', 'data_10_utk', trainable=True)

In [ ]:
model.train()

Epoch 1/40
63/63 [==============================] - 35s 307ms/step - loss: 3.8444 - age_output_loss: 2.9278 - gender_output_loss: 0.9167 - age_output_acc: 0.2488 - gender_output_acc: 0.6794 - val_loss: 2.4613 - val_age_output_loss: 1.9084 - val_gender_output_loss: 0.5529 - val_age_output_acc: 0.2943 - val_gender_output_acc: 0.7590
Epoch 2/40
63/63 [==============================] - 17s 270ms/step - loss: 2.1556 - age_output_loss: 1.7195 - gender_output_loss: 0.4361 - age_output_acc: 0.4083 - gender_output_acc: 0.8198 - val_loss: 2.1649 - val_age_output_loss: 1.6621 - val_gender_output_loss: 0.5028 - val_age_output_acc: 0.3725 - val_gender_output_acc: 0.7922
Epoch 3/40
63/63 [==============================] - 17s 270ms/step - loss: 1.9207 - age_output_loss: 1.5402 - gender_output_loss: 0.3804 - age_output_acc: 0.4581 - gender_output_acc: 0.8473 - val_loss: 2.0638 - val_age_output_loss: 1.5808 - val_gender_output_loss: 0.4830 - val_age_output_acc: 0.4027 - val_gender_output_acc: 0.8217
E

## age_gender_B5', 'data_10_utk'

In [ ]:
NUM_AGE_CLASSES=10
model = Efficient_Net('age_gender_B5', 'data_10_utk', trainable=True)

In [ ]:
model.train()

Epoch 1/40
63/63 [==============================] - 34s 306ms/step - loss: 4.0020 - age_output_loss: 3.0648 - gender_output_loss: 0.9372 - age_output_acc: 0.2347 - gender_output_acc: 0.6691 - val_loss: 2.6114 - val_age_output_loss: 2.0220 - val_gender_output_loss: 0.5894 - val_age_output_acc: 0.2800 - val_gender_output_acc: 0.7010
Epoch 2/40
63/63 [==============================] - 17s 269ms/step - loss: 2.0630 - age_output_loss: 1.6751 - gender_output_loss: 0.3879 - age_output_acc: 0.4146 - gender_output_acc: 0.8276 - val_loss: 2.4794 - val_age_output_loss: 1.9815 - val_gender_output_loss: 0.4979 - val_age_output_acc: 0.4173 - val_gender_output_acc: 0.8180
Epoch 3/40
63/63 [==============================] - 17s 268ms/step - loss: 1.7007 - age_output_loss: 1.4192 - gender_output_loss: 0.2815 - age_output_acc: 0.4767 - gender_output_acc: 0.8784 - val_loss: 2.4415 - val_age_output_loss: 2.1248 - val_gender_output_loss: 0.3168 - val_age_output_acc: 0.4512 - val_gender_output_acc: 0.8690
E

## age_gender_B6', 'data_10_utk'

In [ ]:
NUM_AGE_CLASSES=10
model = Efficient_Net('age_gender_B6', 'data_10_utk', trainable=True)

In [ ]:
model.train()

Epoch 1/40
63/63 [==============================] - 34s 304ms/step - loss: 3.8802 - age_output_loss: 2.9699 - gender_output_loss: 0.9103 - age_output_acc: 0.2494 - gender_output_acc: 0.6752 - val_loss: 2.9241 - val_age_output_loss: 2.3673 - val_gender_output_loss: 0.5568 - val_age_output_acc: 0.3428 - val_gender_output_acc: 0.7642
Epoch 2/40
63/63 [==============================] - 17s 269ms/step - loss: 2.0060 - age_output_loss: 1.6345 - gender_output_loss: 0.3716 - age_output_acc: 0.4341 - gender_output_acc: 0.8382 - val_loss: 7.6148 - val_age_output_loss: 5.4776 - val_gender_output_loss: 2.1372 - val_age_output_acc: 0.3972 - val_gender_output_acc: 0.8410
Epoch 3/40
63/63 [==============================] - 17s 270ms/step - loss: 1.6995 - age_output_loss: 1.4199 - gender_output_loss: 0.2796 - age_output_acc: 0.4776 - gender_output_acc: 0.8785 - val_loss: 1.8169 - val_age_output_loss: 1.4644 - val_gender_output_loss: 0.3525 - val_age_output_acc: 0.4423 - val_gender_output_acc: 0.8443
E

## age_gender_B7', 'data_10_utk'

In [ ]:
NUM_AGE_CLASSES=10
model = Efficient_Net('age_gender_B7', 'data_10_utk', trainable=True)

In [ ]:
model.train()

Epoch 1/40
63/63 [==============================] - 35s 308ms/step - loss: 3.8924 - age_output_loss: 2.9348 - gender_output_loss: 0.9576 - age_output_acc: 0.2474 - gender_output_acc: 0.6720 - val_loss: 2.6380 - val_age_output_loss: 2.1717 - val_gender_output_loss: 0.4663 - val_age_output_acc: 0.3072 - val_gender_output_acc: 0.7878
Epoch 2/40
63/63 [==============================] - 17s 270ms/step - loss: 2.1974 - age_output_loss: 1.7334 - gender_output_loss: 0.4640 - age_output_acc: 0.4162 - gender_output_acc: 0.8147 - val_loss: 2.2335 - val_age_output_loss: 1.8345 - val_gender_output_loss: 0.3991 - val_age_output_acc: 0.3720 - val_gender_output_acc: 0.8248
Epoch 3/40
63/63 [==============================] - 17s 270ms/step - loss: 1.9012 - age_output_loss: 1.5499 - gender_output_loss: 0.3514 - age_output_acc: 0.4661 - gender_output_acc: 0.8578 - val_loss: 2.3874 - val_age_output_loss: 1.9612 - val_gender_output_loss: 0.4262 - val_age_output_acc: 0.3458 - val_gender_output_acc: 0.8260
E